In [ ]:
#!pip3 install --user datasets

In [ ]:
#!pip3 install --user transformers -U

In [1]:
import os

import torch
import torch.nn as nn

from datasets import load_dataset

from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import get_linear_schedule_with_warmup

In [2]:
config = {
    "BATCH_SIZE": 32,
    "NUM_LABEL": 3,
    "NUM_EPOCH": 10,
    "CATEGORY": {
        0: "contradiction",
        1: "neutral",
        2: "entailment"
    }
}
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [4]:
snli = load_dataset("snli")
snli = snli.filter(lambda example: example['label'] != -1)

W0408 16:55:30.316817 140594607580992 builder.py:512] Reusing dataset snli (/usr/users/gpusdi1/gpusdi1_12/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c)
W0408 16:55:30.337524 140594607580992 arrow_dataset.py:1636] Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_12/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-3d6b52b60d37ce7c.arrow
W0408 16:55:30.344197 140594607580992 arrow_dataset.py:1636] Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_12/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c/cache-4d19561d1c41d2b0.arrow
W0408 16:55:30.356946 140594607580992 arrow_dataset.py:1636] Loading cached processed dataset at /usr/users/gpusdi1/gpusdi1_12/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5

In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')

In [6]:
def preprocess(dataset, tokenizer):
    
    def encode(data):
        return tokenizer(data["hypothesis"], data["premise"], truncation=True, padding='max_length')
    
    dataset_pre_processed = dataset.map(encode, batched=True, batch_size=config["BATCH_SIZE"], num_proc=os.cpu_count())
    dataset_pre_processed.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label']) # The model needs tensor as inputs
    dataset_pre_processed = dataset_pre_processed.rename_column("label", "labels") # The model takes as arg labels not label
    
    return dataset_pre_processed["train"], dataset_pre_processed["validation"], dataset_pre_processed["test"] 

In [ ]:
train_data, validation_data, test_data = preprocess(dataset=snli, tokenizer=tokenizer)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(dataset=train_data,
                                              batch_size=config["BATCH_SIZE"],
                                              shuffle=False)
validation_data_loader = torch.utils.data.DataLoader(dataset=validation_data,
                                              batch_size=config["BATCH_SIZE"],
                                              shuffle=False)
test_data_loader = torch.utils.data.DataLoader(dataset=test_data,
                                              batch_size=config["BATCH_SIZE"],
                                              shuffle=False)

In [ ]:
def train(data_loader, model, optimizer, scheduler, device, verbose=False):
    model.train()
    total_number, total_loss = 0, 0.0
    for i, data in enumerate(data_loader):
        if verbose and i%100==0:
            print(f'\t[Training] Progress: {i}/{len(data_loader)}')
        inputs = {
            "input_ids": data["input_ids"].to(device),
            "labels": data["labels"].to(device),
            "attention_mask": data["attention_mask"].to(device)
        }
        outputs = model(inputs["input_ids"], inputs["attention_mask"], labels=inputs["labels"])
        loss = outputs.loss
        loss.backward()
        total_number += inputs["input_ids"].shape[0]
        total_loss += inputs["input_ids"].shape[0] * loss.item()
        
        optimizer.step()
        optimizer.zero_grad()
        
        if i == 1000:
            break
        
    return total_loss/total_number

In [ ]:
def test(data_loader, model, device, verbose=False):
    model.eval()
    total_number, total_loss = 0, 0.0
    with torch.no_grad():
        for i, data in enumerate(data_loader):
            if verbose and i%100==0:
                print(f'\t[Testing] Progress: {i}/{len(data_loader)}')
            inputs = {
                "input_ids": data["input_ids"].to(device),
                "labels": data["labels"].to(device),
                "attention_mask": data["attention_mask"].to(device)
            }
            outputs = model(inputs["input_ids"], inputs["attention_mask"], labels=inputs["labels"])
            loss = outputs.loss
            total_number += inputs["input_ids"].shape[0]
            total_loss += inputs["input_ids"].shape[0] * loss.item()

    return total_loss/total_number

In [ ]:
def train_test(model, train_data_loader, validation_data_loader, test_data_loader, num_epoch, optimizer, scheduler, device, verbose=False):
    for i in range(num_epoch):
        print(f'[Epoch] Epoch number: {i}')
        train_loss = train(train_data_loader, model, optimizer, 
                           scheduler, device, verbose)
        print(f'[Train loss] Train Loss: {train_loss}')
        validation_loss = test(validation_data_loader, model=model, device=device, verbose=verbose)
        print(f'[Validation loss] Validation Loss: {validation_loss}')
        scheduler.step()
    
    test_loss = test(test_data_loader, model, device, verbose=verbose)
    print(f'[Test loss] Test Loss: {test_loss}')

In [ ]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=config["NUM_LABEL"])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = get_linear_schedule_with_warmup(optimizer, 5, 2)

In [ ]:
model.to(device)
for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
train_test(model, train_data_loader, validation_data_loader, test_data_loader, config["NUM_EPOCH"], optimizer, scheduler, device, verbose=True)

In [ ]:
def get_prediction(model, tokenizer, hypothesis, premise, device="cpu"):
    """
        Takes as input the model, a tokenizer, an hypothesis and a premise (by default it will be used on cpu).
        It returns the probabilities for each category.
        It also displays the category.
    """
    inputs = tokenizer(hypothesis, premise, truncation=True, padding='max_length')
    model.eval()
    with torch.no_grad():
        inputs = {
                "input_ids": data["input_ids"].to(device),
                "labels": data["labels"].to(device),
                "attention_mask": data["attention_mask"].to(device)
            }
        outputs = model(inputs["input_ids"], inputs["attention_mask"], labels=inputs["labels"])
        
        logits = outputs.logits
        softmax = nn.Softmax(dim=0)
        probabilities = softmax(logits)[0].items()
        index_max = torch.argmax(probabilities, dim=0)
        prediction = probabilities[index_max]
        
        print(f'{hypothesis} and {premise} are: {config["CATEGORY"][index_max]} with a probability of {prediction}')

    return probabilities

## Example

In [ ]:
model = torch.load("MODEL_PATH", map_location=device)
model.to(device)

In [ ]:
hypothesis = "A soccer game with multiple males playing."
premise = "Some men are playing a sport."
prob = get_prediction(model, tokenizer, hypothesis, premise, device=device)

## VizViz

In [4]:
!pip3 install bertviz --user

     |################################| 163kB 6.0MB/s eta 0:00:01
     |################################| 133kB 15.4MB/s eta 0:00:01
     |################################| 1.2MB 13.7MB/s eta 0:00:01
     |################################| 81kB 6.5MB/s  eta 0:00:01
     |################################| 7.4MB 25.6MB/s eta 0:00:01     |#############                   | 3.1MB 25.6MB/s eta 0:00:01
ERROR: botocore 1.20.47 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.22 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
from bertviz import head_view

In [5]:
#model = torch.load("./models/best_model_aug_V2.pt", map_location=device) # <- Marche pas car pas output attention
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-cased', num_labels=config["NUM_LABEL"], output_attentions=True)
model.to(device)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [10]:
def viz_viz(hypothesis, premise, model, tokenizer, device):
    output_tokenizer = tokenizer(hypothesis, premise, return_tensors="pt", add_special_tokens=True)
    input_ids = output_tokenizer["input_ids"][0].tolist()
    output_tokenizer = output_tokenizer.to(device)
    output_model = model(**output_tokenizer)
    assert "attentions" in output_model, "Model should output attention"
    try:
        attention = output_model["attentions"]
    except KeyError:
        logger.error("Model must ouput attentions")
        return
    input_token = tokenizer.convert_ids_to_tokens(input_ids)
    head_view(attention, input_token)

In [11]:
hypothesis = "Model should output attention"
premise = "Model must ouput attentions"

In [12]:
viz_viz(hypothesis, premise, model, tokenizer, device)

<IPython.core.display.Javascript object>